# Sentiment Analysis - CNN vs LSTM


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:

from nltk import ngrams
import string
 
stop_words=['da','ng','noi','sidi','the','bony','kaiku','tantha','cd','of','and','it','ngna','akhoi','eikhoi','eid','cgi','manipuri','cbu','nupase','nupise','hero','mi','ei','eidi','na','album','bala','c','se','esei','gi','su',]
#stop_words=[] 
# clean words, i.e. remove stopwords and punctuation
def clean_words(words, stop_words):
    words_clean = ''
    for word in words.split():
        word = word.lower()
        if word not in stop_words and word not in string.punctuation:
            words_clean=words_clean+' '+word   
    return words_clean 
 


short_pos = open("pos.txt","r").read()
short_neg = open("neg.txt","r").read()

pos_reviews=[]
neg_reviews=[]
for r in short_pos.split('\n'):
    pos_reviews.append(clean_words(r, stop_words))

for r in short_neg.split('\n'):
    neg_reviews.append(clean_words(r, stop_words))

    
from random import shuffle 
shuffle(pos_reviews)
shuffle(neg_reviews)
train_docs=pos_reviews[:1500]+neg_reviews[:1500]


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import array

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)

max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

ytrain = array([0 for _ in range(1500)] + [1 for _ in range(1500)])

In [ ]:
test_docs = pos_reviews[1500:2000]+neg_reviews[1500:2000]
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define test labels
ytest = array([0 for _ in range(500)] + [1 for _ in range(500)])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

In [ ]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)


In [ ]:
import sklearn.metrics as metrics
import numpy as np

y_pred_labels = model.predict_classes(Xtest, verbose=0)
confusion_matrix = metrics.confusion_matrix(y_true=ytest, y_pred=y_pred_labels)  # shape=(12, 12)
print(confusion_matrix)

In [ ]:
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))


In [ ]:
model = Sequential()
from keras.layers import Dense, Embedding,LSTM
from keras.models import Sequential

# Input / Embdedding
model.add(Embedding(vocab_size,100,input_length=max_length,mask_zero=True))
model.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))

# Output layer
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
epochs = 5
batch_size = 32
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=3, batch_size=64)

In [ ]:
scores = model.evaluate(Xtest, ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
import sklearn.metrics as metrics
import numpy as np

y_pred_labels = model.predict_classes(Xtest, verbose=0)
confusion_matrix = metrics.confusion_matrix(y_true=ytest, y_pred=y_pred_labels)  # shape=(12, 12)
print(confusion_matrix)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
# Fit the model
model.fit(Xtrain, ytrain, epochs=10, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(Xtest, ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
import sklearn.metrics as metrics
import numpy as np

y_pred_labels = model.predict_classes(Xtest, verbose=0)
confusion_matrix = metrics.confusion_matrix(y_true=ytest, y_pred=y_pred_labels)  # shape=(12, 12)
print(confusion_matrix)